In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression

def sig(x):
    a = x<2
    b = x<-2
    return (1-b)*(a*(0.25*x+0.5)+(1-a))

In [2]:
np.set_printoptions(suppress=True)
nb_epoch = 10
nb_users = 800
batch_size = 16
alpha = 0.001

nh = 3
nv = 10
W = np.random.normal(0, 1, (nh, nv))*0.01
a = np.zeros((1, nh))
b = np.ones((1, nv))/2
    

In [3]:
data = np.genfromtxt('data.csv', delimiter=',')
train, train_labels = data[:800,:10], data[:800,-1:]
test, test_labels = data[800:,:10], data[800:,-1:]

train_labels.shape = train_labels.shape[0]
test_labels.shape = test_labels.shape[0]


In [4]:
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = train[id_user:id_user + batch_size]
        v0 = train[id_user:id_user + batch_size]
        ph0 = sig(np.matmul(v0, W.transpose())+a)
        for k in range(3):
            hk = sig(np.matmul(vk, W.transpose())+ a)
            hk = hk > np.random.uniform(0,1,hk.shape)
            vk = sig(np.matmul(hk, W) + b)
            vk = vk > np.random.uniform(0,1,vk.shape)
        phk = sig(np.matmul(vk, W.transpose())+ a)

        dW = np.matmul(v0.transpose(), ph0) - np.matmul(vk.transpose(), phk)
        dW.shape = W.shape
        W += dW*alpha/batch_size
        b += np.sum(v0 - vk, 0) * alpha/batch_size
        a += np.sum(ph0 - phk, 0) * alpha/batch_size
        
        train_loss += np.sum((v0 - vk)**2)
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss))

    train_features = sig(np.matmul(train, W.transpose())+a)
    test_features = sig(np.matmul(test, W.transpose()) + a)

    clf = LogisticRegression()
    clf.fit(train_features, train_labels)
    predictions = clf.predict(test_features)
    print(np.sum(predictions == test_labels), '/', test_labels.shape[0])

epoch: 1 loss: 4200.920068103431
104 / 200
epoch: 2 loss: 4135.783620514574
133 / 200
epoch: 3 loss: 4124.828764742828
169 / 200
epoch: 4 loss: 4029.501960269566
184 / 200
epoch: 5 loss: 3929.1804413017326
188 / 200
epoch: 6 loss: 3996.661844555217
194 / 200
epoch: 7 loss: 3853.0672458949953
198 / 200
epoch: 8 loss: 3878.819524835344
197 / 200
epoch: 9 loss: 3846.366689043508
197 / 200
epoch: 10 loss: 3865.3699627941655
197 / 200
